In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
data = pd.read_json('articles_content_historia.json')
metadata = pd.read_csv('articles_historia.csv')

In [3]:
data.head()
metadata.head()

,title,link,author,date_str
0,"""На Киевском направлении"" – единственный фильм...",https://ukraina.ru/20250610/na-kievskom-naprav...,Алексей Стаценко,16:00
1,"«Добрий вечiр, вам, малята, любi хлопчики й дi...",https://ukraina.ru/20250610/1023859046.html,Василий Стоякин,08:00
2,Мы делили... Черноморский флот,https://ukraina.ru/20250609/1046887656.html,Евгений Антонюк,"Вчера, 16:03"
3,"Король, который не смог стать царём. 430 лет В...",https://ukraina.ru/20250609/1043531930.html,Василий Стоякин,"Вчера, 08:00"
4,Американский счёт Ивана Кожедуба,https://ukraina.ru/20250608/1033029162.html,Дмитрий Заборин,"8 июня, 16:22"


In [4]:
data = pd.merge(data, metadata, left_on='url', right_on='link', how='left', suffixes=('_data', '_metadata'))
print(data.columns)

data['title'] = data.apply(lambda row: row['title_metadata'] if pd.notnull(row['title_metadata']) else row['title_data'], axis=1)
data['author'] = data.apply(lambda row: row['author_metadata'] if pd.notnull(row['author_metadata']) else row['author_data'], axis=1)
data['date_str'] = data.apply(lambda row: row['date_str_metadata'] if pd.notnull(row['date_str_metadata']) else row['date_str_data'], axis=1)

print(data[['url', 'title', 'author', 'date_str']].head())

Index(['title_data', 'link_data', 'author_data', 'date_str_data', 'url',
       'abstract', 'full_text', 'tags', 'views', 'title_metadata',
       'link_metadata', 'author_metadata', 'date_str_metadata'],
      dtype='object')
                                                 url  \
0  https://ukraina.ru/20250610/na-kievskom-naprav...   
1        https://ukraina.ru/20250610/1023859046.html   
2        https://ukraina.ru/20250609/1046887656.html   
3        https://ukraina.ru/20250609/1043531930.html   
4        https://ukraina.ru/20250608/1033029162.html   

                                               title             author  \
0  "На Киевском направлении" – единственный фильм...  Алексей  Стаценко   
1  «Добрий вечiр, вам, малята, любi хлопчики й дi...   Василий  Стоякин   
2                     Мы делили... Черноморский флот   Евгений  Антонюк   
3  Король, который не смог стать царём. 430 лет В...   Василий  Стоякин   
4                   Американский счёт Ивана Кожедуба    Дмитр

In [5]:
data['date_str']

0                       16:00
1                       08:00
2                Вчера, 16:03
3                Вчера, 08:00
4               8 июня, 16:22
                ...          
4341    2 февраля 2016, 15:15
4342    1 февраля 2016, 14:35
4343    23 января 2016, 15:13
4344    22 января 2016, 14:46
4345    20 января 2016, 17:50
Name: date_str, Length: 4346, dtype: object

In [7]:
months = {
    "января": "January", "февраля": "February", "марта": "March", 
    "апреля": "April", "мая": "May", "июня": "June", "июля": "July",
    "августа": "August", "сентября": "September", "октября": "October",
    "ноября": "November", "декабря": "December"
}

def parse_date(date_str):
    if "Вчера" in date_str:
        try:
            time_part = date_str.split(',')[1].strip()
            yesterday = datetime.now() - timedelta(days=1)
            return datetime.strptime(f"{yesterday.strftime('%d %B %Y')}, {time_part}", "%d %B %Y, %H:%M")
        except:
            return None
    if len(date_str.split()) == 1 and ':' in date_str:
        try:
            return datetime.strptime(datetime.now().strftime('%d %B %Y') + " " + date_str, "%d %B %Y %H:%M")
        except:
            return None
    if any(month in date_str for month in months.keys()) and len(date_str.split()) == 3:
        for month_rus, month_eng in months.items():
            if month_rus in date_str:
                date_str = date_str.replace(month_rus, month_eng)
        try:
            date_str = date_str.replace(",", " 2025,")
            return datetime.strptime(date_str, "%d %B %Y, %H:%M")
        except:
            return None
    if any(month in date_str for month in months.keys()) and len(date_str.split()) == 4:
        for month_rus, month_eng in months.items():
            if month_rus in date_str:
                date_str = date_str.replace(month_rus, month_eng)
        try:
            return datetime.strptime(date_str, "%d %B %Y, %H:%M")
        except:
            return None

    return None

data['date_parsed'] = data['date_str'].apply(parse_date)
print(data[['date_str', 'date_parsed']].head())


        date_str         date_parsed
0  17 мая, 16:04 2025-05-17 16:04:00
1  25 мая, 10:21 2025-05-25 10:21:00
2  23 мая, 05:37 2025-05-23 05:37:00
3  28 мая, 08:01 2025-05-28 08:01:00
4  27 мая, 16:00 2025-05-27 16:00:00


In [6]:
null_counts = data.isnull().sum()
print("NB values/columns :")
print(null_counts)

for col in data.columns:
    null_count = null_counts[col]
    if null_count > 0:
        print(f"\n--- Colonne '{col}' contient {null_count} valeurs nulles ---")
        null_rows = data[data[col].isnull()]
        if 'date' in col.lower():
            print(null_rows[['date_str', col]].head(10))
        else:
            print(null_rows[[col]].head(10))


NB values/columns :
title_data              0
link_data               0
author_data          1435
date_str_data           0
url                     0
abstract                0
full_text               0
tags                    0
views                   0
title_metadata       1174
link_metadata        1174
author_metadata      1435
date_str_metadata    1174
title                   0
author               1435
date_str                0
dtype: int64

--- Colonne 'author_data' contient 1435 valeurs nulles ---
    author_data
216        None
238        None
267        None
283        None
304        None
307        None
353        None
358        None
373        None
375        None

--- Colonne 'title_metadata' contient 1174 valeurs nulles ---
     title_metadata
3172            NaN
3173            NaN
3174            NaN
3175            NaN
3176            NaN
3177            NaN
3178            NaN
3179            NaN
3180            NaN
3181            NaN

--- Colonne 'link_metadata' con

In [9]:
def merge_columns(df, new_col, cols_to_merge):
    df[new_col] = None
    for col in cols_to_merge:
        df[new_col] = df[new_col].combine_first(df[col])
    return df

data = merge_columns(data, 'title_merged', ['title', 'title_data', 'title_metadata'])
data = merge_columns(data, 'url_merged', ['url', 'link_data', 'link_metadata'])
data = merge_columns(data, 'author_merged', ['author', 'author_data', 'author_metadata'])
data = merge_columns(data, 'date_str_merged', ['date_str', 'date_str_data', 'date_str_metadata'])


In [10]:
data.head()

,title_data,link_data,author_data,date_str_data,url,abstract,full_text,tags,views,title_metadata,...,author_metadata,date_str_metadata,title,author,date_str,date_parsed,title_merged,url_merged,author_merged,date_str_merged
0,Жёлты воды: первая победа Богдана Хмельницкого,https://ukraina.ru/20250517/1023584796.html,Александр Александров,"17 мая, 16:04",https://ukraina.ru/20250517/1023584796.html,16 мая 1648 года Войско Запорожское нанесло по...,Узнав о начале восстания казаков под руководст...,"[История, История, Богдан Хмельницкий, XVII ве...",12296.0,Жёлты воды: первая победа Богдана Хмельницкого,...,Александр Александров,"17 мая, 16:04",Жёлты воды: первая победа Богдана Хмельницкого,Александр Александров,"17 мая, 16:04",2025-05-17 16:04:00,Жёлты воды: первая победа Богдана Хмельницкого,https://ukraina.ru/20250517/1023584796.html,Александр Александров,"17 мая, 16:04"
1,Фотоальбом Победы: Полтавско-Кременчугская опе...,https://ukraina.ru/20250525/fotoalbom-pobedy-p...,Сергей Зуев,"25 мая, 10:21",https://ukraina.ru/20250525/fotoalbom-pobedy-p...,"В спецпроекте ""Фотоальбом Победы"" мы публикуем...",На главном фото – Полтавско-Кременчугская опер...,"[80 лет Победы, Великая Отечественная война, И...",364.0,Фотоальбом Победы: Полтавско-Кременчугская опе...,...,Сергей Зуев,"25 мая, 10:21",Фотоальбом Победы: Полтавско-Кременчугская опе...,Сергей Зуев,"25 мая, 10:21",2025-05-25 10:21:00,Фотоальбом Победы: Полтавско-Кременчугская опе...,https://ukraina.ru/20250525/fotoalbom-pobedy-p...,Сергей Зуев,"25 мая, 10:21"
2,Фотоальбом Победы: командующий войсками 1-го У...,https://ukraina.ru/20250523/fotoalbom-pobedy-k...,Сергей Зуев,"23 мая, 05:37",https://ukraina.ru/20250523/fotoalbom-pobedy-k...,"В спецпроекте ""Фотоальбом Победы"" мы каждый де...",На первом фото — Командующий войсками 1-го Укр...,"[80 лет Победы, Великая Отечественная война, И...",339.0,Фотоальбом Победы: командующий войсками 1-го У...,...,Сергей Зуев,"23 мая, 05:37",Фотоальбом Победы: командующий войсками 1-го У...,Сергей Зуев,"23 мая, 05:37",2025-05-23 05:37:00,Фотоальбом Победы: командующий войсками 1-го У...,https://ukraina.ru/20250523/fotoalbom-pobedy-k...,Сергей Зуев,"23 мая, 05:37"
3,Армяне и Богдан Хмельницкий,https://ukraina.ru/20250528/1027823642.html,Кирилл Ксенофонтов,"28 мая, 08:01",https://ukraina.ru/20250528/1027823642.html,28 мая в Армении отмечают день республики и эт...,К началу восстания Хмельницкого на современных...,"[История, История, Богдан Хмельницкий, XVII ве...",19357.0,Армяне и Богдан Хмельницкий,...,Кирилл Ксенофонтов,"28 мая, 08:01",Армяне и Богдан Хмельницкий,Кирилл Ксенофонтов,"28 мая, 08:01",2025-05-28 08:01:00,Армяне и Богдан Хмельницкий,https://ukraina.ru/20250528/1027823642.html,Кирилл Ксенофонтов,"28 мая, 08:01"
4,Андреевская церковь: символ имперского Киева,https://ukraina.ru/20250527/1023662362.html,Василий Стоякин,"27 мая, 16:00",https://ukraina.ru/20250527/1023662362.html,27 мая 1754 года было завершено строительство ...,"Сам Растрелли в Киеве, кажется, не работал (ст...","[История, История, Украина, история Украины, а...",6471.0,Андреевская церковь: символ имперского Киева,...,Василий Стоякин,"27 мая, 16:00",Андреевская церковь: символ имперского Киева,Василий Стоякин,"27 мая, 16:00",2025-05-27 16:00:00,Андреевская церковь: символ имперского Киева,https://ukraina.ru/20250527/1023662362.html,Василий Стоякин,"27 мая, 16:00"


In [11]:
columns_to_keep_and_rename = {
    'title_merged': 'title',
    'url_merged': 'url',
    'author_merged': 'author',
}

columns_to_drop = [
    'title', 'title_data', 'title_metadata', 'date_str_merged',
    'url', 'link_data', 'link_metadata', 'date_str'
    'author', 'author_data', 'author_metadata',
    'date_str', 'date_str_data', 'date_str_metadata'
]

data = data.drop(columns=columns_to_drop, errors='ignore')
data = data.rename(columns=columns_to_keep_and_rename)

data['date_parsed'] = pd.to_datetime(data['date_parsed'], errors='coerce')
data = data.loc[:, ~data.columns.duplicated()]

In [12]:
data.head()

,abstract,full_text,tags,views,author,date_parsed,title,url
0,16 мая 1648 года Войско Запорожское нанесло по...,Узнав о начале восстания казаков под руководст...,"[История, История, Богдан Хмельницкий, XVII ве...",12296.0,Александр Александров,2025-05-17 16:04:00,Жёлты воды: первая победа Богдана Хмельницкого,https://ukraina.ru/20250517/1023584796.html
1,"В спецпроекте ""Фотоальбом Победы"" мы публикуем...",На главном фото – Полтавско-Кременчугская опер...,"[80 лет Победы, Великая Отечественная война, И...",364.0,Сергей Зуев,2025-05-25 10:21:00,Фотоальбом Победы: Полтавско-Кременчугская опе...,https://ukraina.ru/20250525/fotoalbom-pobedy-p...
2,"В спецпроекте ""Фотоальбом Победы"" мы каждый де...",На первом фото — Командующий войсками 1-го Укр...,"[80 лет Победы, Великая Отечественная война, И...",339.0,Сергей Зуев,2025-05-23 05:37:00,Фотоальбом Победы: командующий войсками 1-го У...,https://ukraina.ru/20250523/fotoalbom-pobedy-k...
3,28 мая в Армении отмечают день республики и эт...,К началу восстания Хмельницкого на современных...,"[История, История, Богдан Хмельницкий, XVII ве...",19357.0,Кирилл Ксенофонтов,2025-05-28 08:01:00,Армяне и Богдан Хмельницкий,https://ukraina.ru/20250528/1027823642.html
4,27 мая 1754 года было завершено строительство ...,"Сам Растрелли в Киеве, кажется, не работал (ст...","[История, История, Украина, история Украины, а...",6471.0,Василий Стоякин,2025-05-27 16:00:00,Андреевская церковь: символ имперского Киева,https://ukraina.ru/20250527/1023662362.html


In [13]:
data.to_json("data_cleaned_historia.json", orient="records", force_ascii=False, date_format="iso")